# Module 1: Preprocessing and Quality Control

<img src="images/LessonPlan.jpg" alt="Drawing" width="1000" />

## Overview & Purpose
## ChIP-seq, CUT&RUN, and CUT&Tag
This short tutorial demonstrates the initial processing steps for ChIP-seq, CUT&RUN, and CUT&Tag analysis. In this first module, we focus on generating quality reports of the raw sequences, adapter trimming, mapping, and removal of PCR duplicates. 

<img src="images/all3methods.jpg" alt="Drawing" style="width: 500px;"/>

To demonstrate the process, this tutorial will analyze published datasets comparing BAF depletion or mutant to a control sample. This module covers the processing of the data from three distinct but similar methods using downsampled data to improve runtime speed. The original data can be found in the GEO repository with the following accessions: ChIP-seq - p63 occupancy from GSM1645714 & GSM1645715; CUT&RUN - H3K27ac occupancy from GSM4743882 & GSM4743880; CUT&Tag - H3K27ac occupancy from GSM6411302 & GSM6411302. These data were published in the following articles:
Bao X, et al. A novel ATAC-seq approach reveals lineage-specific reinforcement of the open chromatin landscape via cooperation between BAF and p63. Genome Biol 2015 PMID: 26683334
Chang CY, et al. Increased ACTL6A occupancy within mSWI/SNF chromatin remodelers drives human squamous cell carcinoma. Mol Cell 2021 PMID: 34687603
Patil A, et al. A disordered region controls cBAF activity via condensation and partner recruitment. Cell 2023 PMID:  37788668

Note that to allow faster processing we have limited the reads to that of a single chromosome (chr4).  

While ChIP-seq, CUT&RUN, and CUT&Tag are all used to identify chromatin binding sites genome-wide, they differ in implementation which can impact the analysis.
<img src="images/methodcompare.png" alt="Drawing" style="width: 800px;"/>
Image credit: Kaya-Okur et al., 2020. Efficient low-cost chromatin profiling with CUT&Tag. Nature Protocols.

## ChIP-seq
This method uses random fragmentation followed by immunoprecipitation.

<img src="images/chipseq.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&RUN
This method uses random pA-MNase to target the fragmentation to the site where your protein is bound.

<img src="images/CUTRUN.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&Tag
This method uses random pA-Tn5 to directly insert sequencing adapters next to where your protein is bound.

<img src="images/CUTTag.gif" alt="Drawing" style="width: 500px;" align="center"/>


### Ways to use this module
Throughout this module, we have color-coded commands according to ChIP-seq, CUT&RUN, and CUT&Tag. Therefore this module can be used to learn about the processing of each method individually, to compare each method to the others, or you can follow the colored commands to only process one type, either ChIP-seq, CUT&RUN, or CUT&Tag.
Commands for each method will be designated by an individual log before the command, just like the following examples

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [9]:
#run this cell for ChIP-seq
print("Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.")

Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.


<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [10]:
#run this cell for CUT&RUN
print("Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.")

Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.


<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [11]:
#run this cell for CUT&Tag
print("Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.")

Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.


<div class="alert alert-block alert-success" style="font-size:100%">
<span style="color:black"> By following the colors/images, you can run one, two, or all three types of analyses.</span>
</div>

### Required Files
In this stage of the module, you will use the fastq files that have been prepared. You can also use this module on your own data or any published ChIP-seq, CUT&RUN, or CUT&Tag dataset. 

<div class="alert-info" style="font-size:200%">
STEP 1: Set Up Environment
</div>

Initial items to configure your Cloud environment. In this step we will use conda to install the following packages:

Quality Reporting:
[fastqc](https://anaconda.org/bioconda/fastqc), [multiqc](https://anaconda.org/bioconda/multiqc)

Read Trimming: 
[trimmomatic](https://anaconda.org/bioconda/trimmomatic)

Mapping:
[bowtie2](https://anaconda.org/bioconda/bowtie2)

Deduplication:
[samtools](https://anaconda.org/bioconda/samtools), [picard](https://anaconda.org/bioconda/picard)

In [13]:
#First let's install mamba to configure our environment
! curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
! bash Miniforge3-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge
print("done")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 86.0M  100 86.0M    0     0   147M      0 --:--:-- --:--:-- --:--:--  147M
PREFIX=/home/ec2-user/mambaforge

Transaction

  Prefix: /home/ec2-user/mambaforge/envs/_virtual_specs_checks

  All requested packages already installed

Dry run. Not executing the transaction.
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.8.30-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-hf3520f5_7.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.12-5_cp312.conda
Extracting tzdata-2024a-h8827d51_1.conda
Extracting libgomp-14.1.0-h77fa898_1.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-14.1

In [62]:
#now let's install several required packages
!mamba install -c bioconda fastqc bowtie2 picard multiqc samtools trimmomatic deeptools -y
!pip install jupyterquiz==2.0.7 jupytercards
!pip install --user igv-notebook
print("done")


Looking for: ['fastqc', 'bowtie2', 'picard', 'multiqc', 'samtools', 'trimmomatic', 'deeptools']

[+] 0.0s
[+] 0.1s
bioconda/linux-64                                  ━━━╸━━━━━━━━━━━   0.0 B  0.1s
bioconda/noarch                                    ━━━━━━━━━━━━━━━   0.0 B  0.1s
https://aws-ml-conda.s3.us-west-2.amazonaws.com/.. ━━━━━━━━╸━━━━━━   0.0 B  0.1s
https://aws-ml-conda.s3.us-west-2.amazonaws.com/.. ━━━━━━━━━━━━━━━   0.0 B  0.1s
conda-forge/linux-64                               ━━━━╸━━━━━━━━━━   0.0 B  0.1sbioconda/linux-64                                             No change
https://aws-ml-conda.s3.us-west-2.amazonaws.com/..            No change
https://aws-ml-conda.s3.us-west-2.amazonaws.com/..            No change
[+] 0.2s
bioconda/noarch      ╸━━━━━━━━━━━━━━━━━━━━━━ 375.1kB /   4.4MB @   2.1MB/s  0.2s
conda-forge/linux-64 ╸━━━━━━━━━━━━━━━━━━━━━━   3.3MB /  38.6MB @  18.3MB/s  0.2s
conda-forge/noarch   ━━━━━━━━━━╸━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
pytorch/linu

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> The next cell will restart the kernel, so you will see the following message once you run the cell. When this image appears, click okay, and then keep going onto the next cell.</span>
</div>

<img src="./images/restartkernel.jpg" alt="Drawing" style="width: 500px;" align="left"/>

In [ ]:
#Restarts the kernel to implement changes 
import os
os._exit(00)

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> After the kernel restarts continue from here.</span>
</div>

In [49]:
#Now let's import packages that we installed
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
import sys
from jupyterquiz import display_quiz
from IPython.display import IFrame
from IPython.display import display
from jupytercards import display_flashcards
import igv_notebook
import pandas as pd
print("done")

done


In [3]:
# These commands move into our Tutorial 1 directory and create our subdirectory structure.
!mkdir -p Tutorial1/
%cd Tutorial1/
!mkdir -p QC
!mkdir -p Trimmed
!mkdir -p Mapped

/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Tutorial1


In [37]:
#Let's copy and extract our tutorial files
!wget https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/RefGenome.zip
!unzip RefGenome.zip

--2024-10-03 19:51:29--  https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/RefGenome.zip
Resolving chromatinoccupancytutorial.s3.us-east-2.amazonaws.com (chromatinoccupancytutorial.s3.us-east-2.amazonaws.com)... 3.5.133.102, 3.5.131.133, 3.5.128.42, ...
Connecting to chromatinoccupancytutorial.s3.us-east-2.amazonaws.com (chromatinoccupancytutorial.s3.us-east-2.amazonaws.com)|3.5.133.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 354929395 (338M) [application/zip]
Saving to: ‘RefGenome.zip.1’

100%[======================================>] 354,929,395 95.4MB/s   in 3.5s   

2024-10-03 19:51:32 (96.1 MB/s) - ‘RefGenome.zip.1’ saved [354929395/354929395]

Archive:  RefGenome.zip
  inflating: InputFiles/p63ChIPseq_bafi.fastq.gz  
  inflating: InputFiles/p63ChIPseq_ctl.fastq.gz  
  inflating: RefGenome/hg38chr4.1.bt2  
  inflating: RefGenome/hg38chr4.2.bt2  
  inflating: RefGenome/hg38chr4.3.bt2  
  inflating: RefGenome/hg38chr4.4.bt2  
  inflating:

In [38]:
!ls InputFiles

p63ChIPseq_bafi.fastq.gz  p63ChIPseq_ctl.fastq.gz


<div class="alert-info" style="font-size:200%">
STEP 2: QC
</div>

Sequences are typically provided as files in fastq format. This format includes four lines per sequence.


In [39]:
display_flashcards('../flashcards/FastqFlashCard.json')

<IPython.core.display.Javascript object>

### Click on the above image to see what each line represents.
Next, let's take a look at the sequence quality of the raw reads using fastqc:

In [40]:
# This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o QC InputFiles/*fastq.gz

application/gzip
application/gzip


In [41]:
# We can display the resulting fastqc results.
IFrame(src='Tutorial1/QC/p63ChIPseq_ctl_fastqc.html', width=1080, height=800)

<div class="alert-success" style="font-size:100%">
After you've browsed the above report, try to visualize the quality of the other sample(s). Use the cell below to adapt the commands to visualize the bafi sample.
</div>

In [42]:
#Type the code to visualize the fastqc report on the other sample: Tutorial1/QC/p63ChIPseq_bafi_fastqc.html



<div class="alert-info" style="font-size:200%">
Trimming
</div>
<img src="images/trimming.jpg" alt="Drawing" style="width: 250px;"/>
Now that we've viewed the quality let's trim the sequences to remove poor-quality bases and any adapter contamination. We'll use a package called Trimmomatic.

## Let's use trimmomatic to prepare the sequences before mapping.

In [43]:
# This will trim off N's as well as nextera adapters present in the ChIPseq library preparation. Placing the trimmed reads in our Trimmed folder.
!trimmomatic SE -threads $numthreadsint ./InputFiles/p63ChIPseq_ctl.fastq.gz Trimmed/p63ChIPseq_ctl_trimmed.fastq.gz ILLUMINACLIP:RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
#This next command is the same, except we'll do it for the bafi sample.
!trimmomatic SE -threads $numthreadsint ./InputFiles/p63ChIPseq_bafi.fastq.gz Trimmed/p63ChIPseq_bafi_trimmed.fastq.gz ILLUMINACLIP:RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3

print("done")

TrimmomaticSE: Started with arguments:
 -threads 0 ./InputFiles/p63ChIPseq_ctl.fastq.gz Trimmed/p63ChIPseq_ctl_trimmed.fastq.gz ILLUMINACLIP:RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
Automatically using 2 threads
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 525000 Surviving: 525000 (100.00%) Dropped: 0 (0.00%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 0 ./InputFiles/p63ChIPseq_bafi.fastq.gz Trimmed/p63ChIPseq_bafi_trimmed.fastq.gz ILLUMINACLIP:RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRA

<div class="alert-info" style="font-size:200%">
STEP 3: Mapping
</div>
Our fastq files include sequences and quality scores for each base, but we want to figure out which genomic location these sequences came from. To do this we will map each sequence to a reference genome using bowtie2. 
 

Mapping reads requires a reference genome. Due to time and memory considerations, in this tutorial we  prepared that file for you and will only map to chr4. However, in a full analysis, we would map to the entire genome. To do so you would need a fasta file corresponding to the reference genome (e.g. [hg38.fa](https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/)) from which you'd create an index of the genome using bowtie2-build. This can be done with the command: 

bowtie2-build reference_genome_file.fa outputprefix.

As mentioned, we've gone ahead and created the index for you, and, earlier, you copied them into the RefGenome directory. These index files end in the bt2 extension. 

In [44]:
!ls RefGenome/*bt2

RefGenome/hg38chr4.1.bt2  RefGenome/hg38chr4.4.bt2
RefGenome/hg38chr4.2.bt2  RefGenome/hg38chr4.rev.1.bt2
RefGenome/hg38chr4.3.bt2  RefGenome/hg38chr4.rev.2.bt2


<div class="alert-info" style="font-size:100%">
These index files were created from our fasta file:
 </div>

In [50]:
# Notes: The -x option specifies the prefix of the index. -U specifies our fastq file (use -1 and -2 if you have paired-end reads). -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x RefGenome/hg38chr4 -U Trimmed/p63ChIPseq_ctl_trimmed.fastq.gz -S Mapped/p63ChIPseq_ctl.sam
#This runs bowtie2 again, for the other sample.
!bowtie2 -p $numthreadsint -x RefGenome/hg38chr4 -U Trimmed/p63ChIPseq_bafi_trimmed.fastq.gz -S Mapped/p63ChIPseq_bafi.sam

525000 reads; of these:
  525000 (100.00%) were unpaired; of these:
    24855 (4.73%) aligned 0 times
    415471 (79.14%) aligned exactly 1 time
    84674 (16.13%) aligned >1 times
95.27% overall alignment rate
525000 reads; of these:
  525000 (100.00%) were unpaired; of these:
    25062 (4.77%) aligned 0 times
    409400 (77.98%) aligned exactly 1 time
    90538 (17.25%) aligned >1 times
95.23% overall alignment rate


# Bowtie2 outputs a file in [sam format](https://samtools.github.io/hts-specs/SAMv1.pdf), which contains the original sequence, quality scores, and the genomic coordinates matching each read. 

In the next commands we'll convert the file to the more compressed [bam format](https://genome.ucsc.edu/goldenPath/help/bam.html) and sort the reads by chromosomal coordinates.

In [51]:
# This will convert to bam by using samtools view with the -b option. The h and S option tells samtools that the file has a header and is in sam format. We will pipe this to samtools sort. Pay attention to the "-" at the end of the sort command which tells samtools to use stdin.
!samtools view -q 10 -bhS Mapped/p63ChIPseq_ctl.sam | samtools sort -o Mapped/p63ChIPseq_ctl.bam - 
print("done")
#We'll do the same command for our bafi sample.
!samtools view -q 10 -bhS Mapped/p63ChIPseq_bafi.sam | samtools sort -o Mapped/p63ChIPseq_bafi.bam - 
print("done")

done
done


You may have noticed the parameters -bhS and -q 10 in the above commands. Briefly, -bhS describes aspects of the file to samtools, such that you want to output a bam file (the b option), that it has a header (the h option), and that it is currently in sam format (the S option). We also specified -q 10 which removes reads with a mapping score <= 10. 

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 2: Click on the correct answer in the following cell.
</div>

In [52]:
display_quiz("../questions/mappingquality.json")

<IPython.core.display.Javascript object>

<div class="alert-info" style="font-size:200%">
STEP 4: Removal of Duplicates
</div>
It's important to remove duplicates from our reads because part of the ChIP-seq method includes a PCR step for library amplification. This can create biases in the data resulting from PCR duplicates. To understand how PCR duplicates can affect the analysis, let's jump ahead a bit. Occupancy is represented by "peaks" of signal.

<img src="images/peaksofsignal.jpg" alt="Drawing" style="width: 500px;"/>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 3: Click on the correct answer in the following cell.
</div>

In [53]:
display_quiz("../questions/duplicateQuiz.json")

<IPython.core.display.Javascript object>

Okay, let's remove these duplicates using picard.

In [54]:
# This will take the sorted bam file and remove duplicates, saving a new bam file and a summary in a text file.
!picard MarkDuplicates REMOVE_DUPLICATES=TRUE I=Mapped/p63ChIPseq_ctl.bam O=Mapped/p63ChIPseq_ctldedup.bam METRICS_FILE=Mapped/p63ChIPseq_ctldedup_metrics.txt 2> Mapped/PicardLog.txt
print("done")
#Let's do it again, this time for bafi.
!picard MarkDuplicates REMOVE_DUPLICATES=TRUE I=Mapped/p63ChIPseq_bafi.bam O=Mapped/p63ChIPseq_bafidedup.bam METRICS_FILE=Mapped/p63ChIPseq_bafidedup_metrics.txt 2> Mapped/PicardLog.txt
#print("done")

done


<div class="alert-info" style="font-size:200%">
STEP 5: Visualization
</div>
Files in sam/bam format contain a lot of information including the original sequence of the reads, quality scores, and their corresponding chromosomal coordinates.

<img src="images/samformat.jpg" alt="Drawing" style="width: 1000px;"/>


### Please view this [site](https://www.samformat.info/sam-format-flag) for a more complete description of the sam format and to see what the various sam flag values mean.

Let's view the first line of one of our bam files:

In [55]:
!samtools view Mapped/p63ChIPseq_ctldedup.bam | head -n 1 
# Note that there will be an error message because we are breaking a pipe by printing only the first 3 lines. Please ignore the error message.

SRR1944634.7257126	0	chr4	10044	11	50M	*	0	0	CTTAACCCTTAACCCTTAACCCTAACCCTAACCCTAACCCTAACCCTAAC	BBBDFFFFHHHHHIJJJJIJJJJJIGIGGIJIHFGGGGGIIJJIIHGGII	MD:Z:1C48	PG:Z:MarkDuplicates	XG:i:0	NM:i:1	XM:i:1	XN:i:0	XO:i:0	AS:i:-5	XS:i:-13	YT:Z:UU
samtools view: writing to standard output failed: Broken pipe
samtools view: error reading file "Mapped/p63ChIPseq_ctldedup.bam": Broken pipe
samtools view: error closing standard output: -1


While we can see the coordinates of each read, we will need a better way of visualizing the results. In this step we will create a binary file that summarizes the pileup of reads at each base-pair along our genome, in [bigwig](http://genome.ucsc.edu/goldenPath/help/bigWig.html) format. 

To create the bigwig files let's use the command bamCoverage, part of the [deeptools](https://deeptools.readthedocs.io/en/develop/) package.

In [63]:
# First we need to create an index of our bam file.
!samtools index Mapped/p63ChIPseq_ctldedup.bam

# Then we can create a bigwig file of the control sample.
!bamCoverage -b Mapped/p63ChIPseq_ctldedup.bam -o Mapped/p63ChIPseq_ctldedup.bw -bs 1 -p $numthreadsint --normalizeUsing BPM 2> Mapped/bamCovLog_ctl.txt

# Now let's rerun the commands for our other sample.
!samtools index Mapped/p63ChIPseq_bafidedup.bam
!bamCoverage -b Mapped/p63ChIPseq_bafidedup.bam -o Mapped/p63ChIPseq_bafidedup.bw -bs 1 -p $numthreadsint --normalizeUsing BPM 2> Mapped/bamCovLog_bafi.txt

print("done")

done


In the above example we specify the bam file name after -b and the output file name after -o. 

We specified -bs 1, which tells bamCoverage to summarize the reads at every basepair.

We also specified the number of threads to use with -p, which is held in a variable in our notebook.

Lastly, we specified --normalizeUsing BPM. BPM stands for Bins Per Million mapped reads. What do you think this normalization does?

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Click on the correct answer in following cell.
</div>

In [64]:
display_quiz("../questions/BPMnorm.json")

<IPython.core.display.Javascript object>

<div class="alert-info" style="font-size:150%">
Genome Browser
</div>

Now that we have our bigwig files, we can visualize the signal in a genome browser. We'll use [igv](https://igv.org/) in this example.

In [65]:
igv_notebook.init()
myigv = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr4:55,400,000-56,400,000"
    }
)
myigv.load_track(
{
        "name": "p63ChIPseq_ctl",
        "url": "Tutorial1/Mapped/p63ChIPseq_ctldedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "p63ChIPseq_bafi",
        "url": "Tutorial1/Mapped/p63ChIPseq_bafidedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This will load in the signal into IGV and allow you to browse the genome. Feel free to play around with this. More instructions can be found on the [IGV](https://igv.org/) website.

Click the "gear" icon on the right of each track to customize the color, name, height, etc... 

The default is to autoscale the tracks. But this means that our samples have different y-axis scales! Because we BPM normalized, we should set the data range to be the same for each sample.

<img src="images/igvgear.jpg" alt="Drawing" style="width: 1000px;"/>

<div class="alert alert-block alert-success" style="font-size:200%">
<span style="color:black">Congrats! You have successfully preprocessed the samples and can visually browse the results. In the next tutorial, we will continue with the processing by identifying peaks and performing differential analysis.</span>
</div>
